In [1]:
import numpy as np
import pandas
import matplotlib.pyplot as plt

import scipy.stats

In [2]:
import sys
import requests
import importlib

def import_local_or_github(package_name, function_name=None, directory=None, giturl=None):
    # Import functions directly from github
    # Important: note that we use raw.githubusercontent.com, not github.com

    try: # to find the file locally
        if directory is not None:
            if directory not in sys.path:
                sys.path.append(directory)
    
        package = importlib.import_module(package_name)
        if function_name is not None:
            function = getattr(package, function_name)
            return function
        else:
            return package

    except: # get the file from github
        if giturl is None:
            giturl = 'https://raw.githubusercontent.com/florisvb/Nonlinear_and_Data_Driven_Estimation/main/Utility/' + str(package_name) + '.py'
        
        r = requests.get(giturl)
        print('Fetching from: ')
        print(r)
    
        # Store the file to the colab working directory
        with open(package_name+'.py', 'w') as f:
            f.write(r.text)
        f.close()
    
        # import the function we want from that file
        package = importlib.import_module(package_name)
        if function_name is not None:
            function = getattr(package, function_name)
            return function
        else:
            return package

planar_drone = import_local_or_github('planar_drone', directory='../Utility')
plot_tme = import_local_or_github('plot_utility', 'plot_tme', directory='../Utility')

/home/caveman/PY38/lib/python3.8/site-packages/do_mpc/sysid/__init__.py:15: UserWarning: The ONNX feature is not available. Please install the full version of do-mpc to access this feature.
  warnings.warn('The ONNX feature is not available. Please install the full version of do-mpc to access this feature.')
/home/caveman/PY38/lib/python3.8/site-packages/do_mpc/opcua/__init__.py:14: UserWarning: The opcua feature is not available. Please install the full version of do-mpc to access this feature.
  warnings.warn('The opcua feature is not available. Please install the full version of do-mpc to access this feature.')


# Load the data

And format it so the shapes and organization of the data matches standard mathematical notation.

### Load the arrays for the kalman filter

Colab tip: You may need to download them to your machine from notebook A, and then upload them to the colab environment where you are running this notebook.

In [3]:
# read from disk
loaded_arrays = np.load('ABCDRQ.npz')
A = loaded_arrays['A']
B = loaded_arrays['B']
C = loaded_arrays['C']
D = loaded_arrays['D']
R = loaded_arrays['R']
Q = loaded_arrays['Q']

In [4]:
print('A shape: ', A.shape)
print('B shape: ', B.shape)
print('C shape: ', C.shape)
print('D shape: ', D.shape)
print('R shape: ', R.shape)
print('Q shape: ', Q.shape)

A shape:  (7, 7)
B shape:  (7, 2)
C shape:  (4, 7)
D shape:  (4, 2)
R shape:  (4, 4)
Q shape:  (7, 7)


### Load the state data and reorganize Y and U

Standard mathematical notation requires that each column of $Y$ correspond to the measurements at discrete time, i.e. $\mathbf{y}_k$, and each row of $\mathbf{y}$ correspond to each individual measurement. Similarly for $U$. 

In [5]:
# read from disk
df_states = pandas.read_hdf('planar_drone_states.hdf')
df_measurements_true = pandas.read_hdf('planar_drone_measurements_true.hdf')
df_measurements_noisy = pandas.read_hdf('planar_drone_measurements_noisy.hdf')

In [6]:
Y = np.vstack(df_measurements_noisy[['theta', 'x', 'z', 'k']].values).T
print(Y.shape)

(4, 200)


In [7]:
U = np.vstack(df_states[['j1', 'j2']].values).T
print(U.shape)

(2, 200)


### Tip: When you index Y or U, make sure you are keeping a column vector. 

In [8]:
# This is wrong
y_0 = Y[:, 0]
print(y_0)

[0.22321732 0.81268371 0.63948923 0.99070634]


In [9]:
# This is right
y_0 = Y[:, [0]]
print(y_0)

[[0.22321732]
 [0.81268371]
 [0.63948923]
 [0.99070634]]


## Tip: matrix multiplication in python

Recall that matrix shape during multiplication works like: 

$[N\times M][M\times 1]=[N\times 1]$

In [15]:
# Option 1 using arrays
print('R shape: ', R.shape)
print('y_0 shape: ', y_0.shape)
print('[R x y_0] shape', (R@y_0).shape)
print('')
print('[R x y_0] = ')
R@y_0

R shape:  (4, 4)
y_0 shape:  (4, 1)
[R x y_0] shape (4, 1)

[R x y_0] = 


array([[2.23217318e-03],
       [3.25073486e-02],
       [1.02318277e-01],
       [9.90706342e-05]])

In [16]:
# Option 2 using matrices -- NOTE: np.matrix is getting deprecated, sadly, so this is not recommended
R_times_y_0 = np.matrix(R)*np.matrix(y_0)
print('[R x y_0] = ')
R_times_y_0

[R x y_0] = 


matrix([[2.23217318e-03],
        [3.25073486e-02],
        [1.02318277e-01],
        [9.90706342e-05]])

# Exercises:

1. Code a linear discrete Kalman filter that uses the matrices and time series data defined above:
    * $A, B, C, D, R, Q$ matrices defined above
    * Measurements are given by $Y$
    * Controls are given by $U$
    * Choose an initial $\mathbf{x_0}$ and $P_0$
    * Try a larger value of Q, and a smaller value of Q. For this problem, does a larger or smaller value of Q lead to more accurate estimates? Why do you think that is?

2. Compare the error covariance of your Kalman filter to the Cramer Rao Bound. Is your KF efficient? Try for different values of Q.

3. Compare the error covariance of your KF and the CRB to the actual errors in your estimate. Does the distribution of the errors match that predicted by the error covariance and the CRB? Hint: the error covariance and the CRB report variance, the square of the standard deviation. 